In [1]:
import pandas as pd
import string
import numpy as np
import nltk

In [2]:
data=pd.read_csv("SPAM_text.csv")
data['labels']=np.nan
data.head()

,Category,Message,labels
0,ham,"Go until jurong point, crazy.. Available only ...",NaN
1,ham,Ok lar... Joking wif u oni...,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN
3,ham,U dun say so early hor... U c already then say...,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN


In [3]:
classes = {
    "ham":0,
    "spam":1,
}

data['labels'] = data['Category'].map(classes)

In [4]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

In [5]:
def remove_stopwords(text):
    text = ''.join(word.lower() for word in text if word not in string.punctuation)
    tokens = text.split()
    text = [word for word in tokens if word  not in stopwords]
    return text

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer= remove_stopwords)

In [7]:
X = vectorizer.fit_transform(data['Message'])
X

<5572x9526 sparse matrix of type '<class 'numpy.int64'>'
	with 50053 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, data['labels'], test_size = 0.2, random_state = 42)

In [9]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=4)
nb.fit(x_train, y_train)

MultinomialNB(alpha=4)

In [10]:
y_pred_nb = nb.predict(x_test)

In [11]:
y_pred_nb

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [12]:
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score
classification_report = classification_report(y_test, y_pred_nb)
print(accuracy_score(y_test, y_pred_nb))

0.9775784753363229


In [13]:
print(classification_report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       0.97      0.86      0.91       149

    accuracy                           0.98      1115
   macro avg       0.97      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [19]:
text_test="Hi Customer, You're Pre-Qualified for a No Documentation personal loan from L&T Finance. Instant Disbursal. Apply Now! oow.pw/k0A2o6 SPIRAEA"

In [20]:
bc=vectorizer.transform([text_test])
if nb.predict(bc)[0] == 0:
    print("Not a spam Message")
else:
    print("Spam message")

Spam message


In [16]:
nb.get_params()

{'alpha': 4, 'class_prior': None, 'fit_prior': True}

In [34]:
text2="Dear K,get quick funds with a pre-approved Personal Loan of upto 7Lakh from L&T Finance. To APPLY CLICK https://planet.ltfs.com/LTFS-CL/ "

In [35]:

from sklearn.model_selection import GridSearchCV

params = {'alpha': [6,4,1,1e-1,1e-2,1e-3],
          'fit_prior': [True, False]}

grid_search = GridSearchCV(nb, params, scoring='accuracy',cv=9,n_jobs=-1)
grid_search.fit(x_train, y_train)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

MultinomialNB(alpha=4)
0.9762146555157307


In [36]:
grid_search.predict(vectorizer.transform([text_test]))
if nb.predict(vectorizer.transform([text2]))[0] == 0:
    print("Not a spam Message")
else:
    print("Spam message")

Spam message
